In [19]:
import os
import sys

import warnings
warnings.filterwarnings("ignore")

os.environ['environment'] = 'interactive'
os.environ['UTILS_PATH'] = '/Users/arsenijkadaner/Yandex.Disk.localized/master/code/git/python-utils-ak'
os.environ['PROJECT_PATH'] = '/Users/arsenijkadaner/Yandex.Disk.localized/master/code/git/2020.10-umalat/umalat' 
os.environ['SQLITE_PATH'] = '/Users/arsenijkadaner/Yandex.Disk.localized/master/code/git/2020.10-umalat/umalat/data.sqlite' 
sys.path.append(os.environ['UTILS_PATH'])
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

In [116]:
from utils_ak.block_tree import *
from utils_ak.iteration import *
from utils_ak.builtin import *

from app.schedule_maker.models import *

In [171]:
sku = cast_model(CreamCheeseSKU, 91)
values = [[0, sku, 10]]
boiling_group_df = pd.DataFrame(values, columns=["boiling_id", "sku", "kg"])
boiling_group_df

,boiling_id,sku,kg
0,0,<CreamCheeseSKU 91>,10


In [175]:
def make_cream_cheese_boiling(boiling_group_df):
    boiling_model = boiling_group_df.iloc[0]["sku"].made_from_boilings[0]
    
    boiling_id = boiling_group_df.iloc[0]["boiling_id"]
    maker, make = init_block_maker(
        "boiling", boiling_model=boiling_model, boiling_id=boiling_id
    )
    
    bt = boiling_model.boiling_technology

    with make("boiling_process"):
        make("pouring", size=(bt.cooling_time // 5, 0))
        make("separation", size=(bt.separation_time // 5, 0))
        make("salting", size=(bt.salting_time // 5, 0))
        make("separation", size=(bt.separation_time // 5, 0))
        make("salting", size=(bt.salting_time // 5, 0))
        make("P", size=(bt.p_time // 5, 0))
    with make("packing_process", x=(maker.root["boiling_process"]['separation'][-1].x[0] - bt.p_time // 5, 0), push_func=add_push):
        make("P", size=(bt.p_time // 5, 0))

        packing_time = sum(
            [
                row["kg"] / row["sku"].packing_speed * 60
                for i, row in boiling_group_df.iterrows()
            ]
        )
        packing_time = int(custom_round(packing_time, 5, "ceil"))

        make(
            "packing",
            size=(packing_time // 5, 0),
        )
    return maker.root

In [176]:
make_cream_cheese_boiling(boiling_group_df)

====================================================== boiling (0, 54] x (0, 0]
============================================== boiling_process (0, 46] x (0, 0]
============ pouring (0, 12] x (0, 0]
            ============ separation (12, 24] x (0, 0]
                        ==== salting (24, 28] x (0, 0]
                            ============ separation (28, 40] x (0, 0]
                                        ==== salting (40, 44] x (0, 0]
                                            == P (44, 46] x (0, 0]
                          ============================ packing_process (26, 54] x (0, 0]
                          == P (26, 28] x (0, 0]
                            ========================== packing (28, 54] x (0, 0]

In [ ]:
def make_cream_cheese_boiling_group(boiling_group_dfs):
    boiling_model = boiling_group_df1.iloc[0]["sku"].made_from_boilings[0]

    validator = ClassValidator(window=3)

    def validate(b1, b2):
        validate_disjoint_by_axis(
            b1["packing_process"]["packing"], b2["packing_process"]["packing"]
        )
        # just in case, not needed in reality
        validate_disjoint_by_axis(
            b1["boiling_process"]["separation"], b2["boiling_process"]["separation"]
        )
        assert (
            b2["boiling_process"]["pouring"].x[0]
            >= b1["boiling_process"]["pouring"].y[0]
        )

    validator.add("boiling", "boiling", validate)

    maker, make = init_block_maker(
        "mascarpone_boiling_group", boiling_model=boiling_model
    )

    b1 = make_mascorpone_boiling(boiling_group_df1)
    b2 = make_mascorpone_boiling(boiling_group_df2)
    for b in [b1, b2]:
        push(
            maker.root,
            b,
            push_func=AxisPusher(start_from="last_beg"),
            validator=validator,
        )
    # fix waiting time
    waiting_size = (
        b2["boiling_process"]["pouring"].x[0] - b1["boiling_process"]["pouring"].y[0]
    )

    for b in [b2["boiling_process"]]:
        b.props.update(x=(b.props["x_rel"][0] - waiting_size, 0))
    for key in ["adding_lactic_acid", "separation"]:
        b = b2["boiling_process"][key]
        b.props.update(x=(b.props["x_rel"][0] + waiting_size, 0))
    b2["boiling_process"]["waiting"].props.update(size=(waiting_size, 0))
    return maker.root